The goal of this work is to implement, and test a method program that divides a dataset to a number of (maybe) overlapping subsets.
- Dataset 
- Distance 
- Homogeneity
- Decomposition
 - Baseline 
 - Our method

The link shared in Blackbord:
https://colab.research.google.com/drive/1tvxs0etNUDw1_8Aeu5G6y6qOIBROq-G6?usp=sharing#scrollTo=B7Gj19ZxzJGO


In [ ]:
# only run this once in local environment
!wget https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar -xvf spark-3.2.1-bin-hadoop3.2.tgz
!java -version
!pip install findspark

In [1]:
import os 
import findspark
import string
import urllib.request
import pandas as pd

from pyspark.sql import SparkSession， Row
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql import SQLContext, SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import Word2Vec
from pyspark.sql.functions import *


os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"
# put your own directory here
# os.environ["SPARK_HOME"] = "/home/com3dian/Documents/github/Period4/DIS/spark-3.2.1-bin-hadoop3.2"
findspark.init()

In [2]:
# read data
url="https://raw.githubusercontent.com/mustafa-majid/disfiles/main/cities.csv"
pdf=pd.read_csv(url)
pdf.head(5)

In [4]:
p2=pd.read_csv(url).drop_duplicates()
p2.head(5)

In [6]:
file = urllib.request.urlopen(url)
print(type(file))
li=[]
for line in file:
    s = line.decode("utf-8").translate(str.maketrans('', '', string.punctuation))
    li.append(" ".join(s.split()))

li.pop(0)
print(li[2])
print(type(li[2]))

<class 'http.client.HTTPResponse'>
46 35 59 N 120 30 36 W Yakima WA
<class 'str'>


In [8]:
sc = SparkContext()
rdd1 = sc.parallelize(li)
row_rdd = rdd1.map(lambda x: Row(x))
sqlContext = SQLContext(sc)

df=sqlContext.createDataFrame(row_rdd, ['words'])
df.show(50, False)

/home/com3dian/Documents/github/Period4/DIS/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+----------------------------------------+
|words                                   |
+----------------------------------------+
|41 5 59 N 80 39 0 W Youngstown OH       |
|42 52 48 N 97 23 23 W Yankton SD        |
|46 35 59 N 120 30 36 W Yakima WA        |
|42 16 12 N 71 48 0 W Worcester MA       |
|43 37 48 N 89 46 11 W Wisconsin Dells WI|
|36 5 59 N 80 15 0 W WinstonSalem NC     |
|49 52 48 N 97 9 0 W Winnipeg MB         |
|39 11 23 N 78 9 36 W Winchester VA      |
|34 14 24 N 77 55 11 W Wilmington NC     |
|39 45 0 N 75 33 0 W Wilmington DE       |
|48 9 0 N 103 37 12 W Williston ND       |
|41 15 0 N 77 0 0 W Williamsport PA      |
|37 40 48 N 82 16 47 W Williamson WV     |
|33 54 0 N 98 29 23 W Wichita Falls TX   |
|37 41 23 N 97 20 23 W Wichita KS        |
|40 4 11 N 80 43 12 W Wheeling WV        |
|26 43 11 N 80 3 0 W West Palm Beach FL  |
|47 25 11 N 120 19 11 W Wenatchee WA     |
|41 25 11 N 122 23 23 W Weed CA          |
|31 13 11 N 82 20 59 W Waycross GA       |
|44 57 35 N

In [9]:
df2 = df.withColumn("text_splitted", split(lower(col("words")), " "))
df2.show(20)

In [18]:
# warnings to fix
word2Vec = Word2Vec(minCount=0, 
                    maxIter=100, 
                    inputCol="text_splitted", 
                    outputCol="features")
word2Vec.setVectorSize(50)
model = word2Vec.fit(df2)

Exception ignored in: <function JavaWrapper.__del__ at 0x7f9c77651040>
Traceback (most recent call last):
  File "/home/com3dian/Documents/github/Period4/DIS/spark-3.2.1-bin-hadoop3.2/python/pyspark/ml/wrapper.py", line 39, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'Word2Vec' object has no attribute '_java_obj'


In [19]:
result = model.transform(df2)

In [20]:
numIterations = 300
numberClusters = 30
kmeans = KMeans().setMaxIter(numIterations).setK(numberClusters).setSeed(1)
kmeans_model = kmeans.fit(result)

In [21]:
from pyspark.sql.functions import col

predictions = kmeans_model.transform(result)
predictions=predictions.orderBy(predictions.prediction.desc())
predictions = predictions.drop(col("features"))
predictions = predictions.drop(col("text_splitted"))
predictions.show(100,False)


+-------------------------------------------+----------+
|words                                      |prediction|
+-------------------------------------------+----------+
|32 25 11 N 87 1 11 W Selma AL              |29        |
|44 45 35 N 85 37 47 W Traverse City MI     |29        |
|38 53 23 N 77 1 47 W Washington DC         |29        |
|44 56 23 N 123 1 47 W Salem OR             |29        |
|38 31 47 N 106 0 0 W Salida CO             |28        |
|37 10 11 N 104 30 36 W Trinidad CO         |28        |
|33 24 0 N 104 31 47 W Roswell NM           |28        |
|50 25 11 N 104 39 0 W Regina SA            |28        |
|43 25 48 N 83 56 24 W Saginaw MI           |27        |
|35 40 48 N 105 56 59 W Santa Fe NM         |27        |
|38 42 35 N 93 13 48 W Sedalia MO           |26        |
|37 13 12 N 93 17 24 W Springfield MO       |26        |
|41 35 24 N 109 13 48 W Rock Springs WY     |26        |
|41 32 59 N 73 3 0 W Waterbury CT           |25        |
|42 49 11 N 73 56 59 W Schenect

In [22]:
model.getVectors().show(20,False)

+-------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [23]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

similar("Christian", "Christine")

0.8888888888888888